# Introduction

### This project report will include
1. Explanation of the purpose, and background on my project
2. A in-depth explanation of my data merging


## Purpose
The purpose of this project report (in notebook form) is to show some exploratory data analysis and data cleaning for this particular project. This project set out to reveal information about the relationship between population demographic information and the uptake in solar power. In particular, the purpose was to determine whether race or income was more significant in solar power uptake. 

## Background
The initial inspiration for this project was a Graduate Mechanical Engineering seminar at Tufts University. The talk, given on Thursday February 7th, 2019, was titled titled “Energy Injustice: Disparities in Rooftop Photovoltaics Deployment in the United States.” The abstract of the talk was as follows: 

“The rooftop solar industry in the United States has experienced dramatic growth—roughly 50% per year since 2012, along with steadily falling prices. Although the opportunities this affords for clean, reliable power are transformative, the benefits might not accrue to all individuals and communities. Combining the location of existing and potential sites for rooftop photovoltaics (PV) from Google’s Project Sunroof and demographic information from the American Community Survey, the relative adoption of rooftop PV is compared across census tracts based on median household income, home ownership, and race and ethnicity. The historical context of energy injustice in this country will be discussed, as well as the importance for engineers today to holistically consider sustainability: environmentally, economically, and socially.”

During the talk, Professor Sunter detailed how she ran regressions on data sets from the American Community Survey (ACS) and Google’s Project Sunroof program to determine the factor that most significantly affects solar power uptake. Since the ACS, which is administered by the US Census Bureau, ties geography to certain population characteristics (e.g. race, income, and age), and the Project Sunroof data set details solar power characteristics (e.g. number of solar panels), she was able statistically detail the effect of race on solar deployment. Since both data sets provided information at the census tract level, that was chosen as the geographic level of interest.  

As a first foray into data science techniques, I will try to replicate the work that Professor Sunter has done. In her paper, “Disparities in rooftop photovoltaics deployment in the United States by race and ethnicity”, Professor Sunter details her methods for determining if there are significant differences between races in solar deployment, all else equal. One significant reason for replication of this study is that the ACS data set that was utilized in the study came from the 2009-2013 5-year ACS. In my project, I will be using the 2012-2017 5-year ACS data set, providing a much more recent set of population characteristics. 

Solar power companies have approached Professor Sunter about the results of her data analysis to determine ways to equalize, from a racial point of view, access and usage of solar power. Solar power has the potential to pay for itself, reducing or eliminating the electricity bill of its users, with the caveat of significant policy differences between states. 


## Initialization
Below are some of the tools that I used to conduct the data exploration and  visuaization. 

In [10]:
import requests
import pandas as pd
import seaborn as sns
import geopandas as gpd
import numpy as np
from census import Census
from us import states
import time
import os
import re

## Acquiring Datasets
There are two data sets that are read into the analysis. The first is the American Community Survey (ACS), part of the US Census's collected data. The particular ACS data that is read in is the 5-year aggregate collected in the years 2013-2017. Using the "census" wrapper (as was suggested by the official Census website), the cell below scrapes the data set, collecting information on total population of all counties. 

I then decided that I would focus mainly on the most populous counties in the US, as a proxy for the popularity of solar panels. This idea was inspired by Google Project Sunroof's machine learning algorithm. The team fed the algorithm mainly aerial imagery from areas that were more dense in their solar panel coverage in order to expose the algorithm to more positive hits. In the code, I sorted all the counties in the US by total county population and then chose a number of those most populous counties to continue on with.

Using the Census wrapper, a dataframe called censusdata was built, collecting data on total population, racial demographic information, and income on each of the census tracts in each of those very populous counties. Some data cleaning occurs in the block of code, there are some census tracts with very low populations that were excluded and there were also some negative income values for some census tracts. These census tracts were dropped from the analysis. 

In [11]:
start = time.time()
c = Census("03b8bb083da55549598393f0f23156a48c2c0c85", year=2017)
county = c.acs5.get(('NAME', 'B01003_001E'), {'for':'county:*'})
df = pd.DataFrame(county)
countysort = df.sort_values(["B01003_001E"], axis = 0, ascending = False)
countysortshort = countysort.head(5)
css = countysortshort

#df2 = pd.DataFrame(c.acs5.state_county_tract(('NAME', 'B01003_001E','B02001_002E', 'B02001_003E', 'B02001_005E', 'B03002_012E', 'B19013_001E'), css.iloc[0,3], css.iloc[0,2], Census.ALL))
countyData = pd.DataFrame(c.acs5.state_county_tract(('NAME', 'B01003_001E','B02001_002E', 'B02001_003E', 'B02001_005E', 'B03002_012E', 'B19013_001E'), css.iloc[0,3], css.iloc[0,2], Census.ALL))
for x in range(1,len(css)):
    df = pd.DataFrame(c.acs5.state_county_tract(('NAME', 'B01003_001E','B02001_002E', 'B02001_003E', 'B02001_005E', 'B03002_012E', 'B19013_001E'), css.iloc[x,3], css.iloc[x,2], Census.ALL))
    countyData = countyData.append(df, ignore_index = True)
    #print("I am at step ", x)
print('Out of Loop')

countyData['tractID'] = countyData['state'].str.cat(countyData['county'],sep="").str.cat(countyData['tract'], sep="")
censusdata = countyData
censusdata.tractID = censusdata.tractID.astype(float)
censusdata.columns = ['TotalPopulation', 'White', 'Black', 'Asian', 'Hispanic','Income', 'NAME', 'county', 'state', 'tract', 'tractID']
censusdata = censusdata.drop(['county', 'state', 'tract'], axis = 1)
censusdatasort = censusdata.sort_values(["tractID"], axis = 0, ascending = True)
#censusdatasort
censusData = censusdatasort.drop(censusdatasort[censusdatasort.TotalPopulation<1500].index)
censusData = censusData.drop(censusData[censusData.Income<0].index)
end = time.time()
total = end-start
print('Operation took', total, 'seconds')
censusData.head()

Out of Loop
Operation took 5.904295206069946 seconds


,TotalPopulation,White,Black,Asian,Hispanic,Income,NAME,tractID
4880,4915.0,4814.0,0.0,19.0,78.0,87167.0,"Census Tract 101.01, Maricopa County, Arizona",4.013010e+09
4902,4602.0,4408.0,25.0,134.0,59.0,115725.0,"Census Tract 101.02, Maricopa County, Arizona",4.013010e+09
4985,4539.0,4472.0,0.0,62.0,170.0,113889.0,"Census Tract 304.01, Maricopa County, Arizona",4.013030e+09
4905,4243.0,4031.0,17.0,0.0,121.0,81994.0,"Census Tract 304.02, Maricopa County, Arizona",4.013030e+09
4468,5110.0,4762.0,0.0,4.0,834.0,40434.0,"Census Tract 405.02, Maricopa County, Arizona",4.013041e+09


## More Data Import: Google Project Sunroof
The next few lines simply put the data from Google Project sunroof into its own dataframe. Additionally, the column that the Sunroof data had called region_name, which stored its own census tract number information, was converted to floats (rather than object data types) and renamed tractID. This is to facilitate the merging of the two datasets using the 'merge' command in pandas. 



In [12]:
start = time.time()
sunroofData = pd.DataFrame(pd.read_csv('https://storage.googleapis.com/project-sunroof/csv/latest/project-sunroof-census_tract.csv'))
sunroofData.region_name = sunroofData.region_name.astype(float)
sunroofData.rename(columns={'region_name':'tractID'}, inplace=True)
end = time.time()
duration = end-start
print('Operation took', duration, 'seconds')
sunroofData.head()

Operation took 8.085880517959595 seconds


,tractID,state_name,lat_max,lat_min,lng_max,lng_min,lat_avg,lng_avg,yearly_sunlight_kwh_kw_threshold_avg,count_qualified,...,yearly_sunlight_kwh_n,yearly_sunlight_kwh_s,yearly_sunlight_kwh_e,yearly_sunlight_kwh_w,yearly_sunlight_kwh_f,yearly_sunlight_kwh_median,yearly_sunlight_kwh_total,install_size_kw_buckets_json,carbon_offset_metric_tons,existing_installs_count
0,1.001020e+09,Alabama,32.505161,32.449730,-86.475128,-86.510422,32.477184,-86.490082,1095.65,591.0,...,6.735739e+05,3.744787e+06,2.558342e+06,2.776739e+06,2.567009e+06,14854.402856,1.232045e+07,"[[0,76],[5,163],[10,136],[15,78],[20,52],[25,2...",7325.568357,0.0
1,1.001020e+09,Alabama,32.489342,32.456120,-86.465378,-86.481270,32.474255,-86.473381,1095.65,718.0,...,1.354248e+06,4.702247e+06,2.993705e+06,2.978384e+06,6.819697e+06,13564.145190,1.884828e+07,"[[0,122],[5,200],[10,168],[15,89],[20,46],[25,...",11206.925804,0.0
2,1.001020e+09,Alabama,32.491909,32.459209,-86.450684,-86.470871,32.475426,-86.460190,1095.65,1121.0,...,2.010001e+06,7.390340e+06,3.998881e+06,4.096211e+06,8.226100e+06,12725.930640,2.572153e+07,"[[0,167],[5,363],[10,258],[15,150],[20,61],[25...",15293.664803,1.0
3,1.001020e+09,Alabama,32.493690,32.447208,-86.432381,-86.453941,32.472000,-86.443619,1095.65,1491.0,...,2.840030e+06,1.006799e+07,5.554514e+06,6.354775e+06,4.816764e+06,16586.321436,2.963407e+07,"[[0,174],[5,332],[10,350],[15,285],[20,185],[2...",17620.007964,0.0
4,1.001020e+09,Alabama,32.485748,32.424000,-86.411346,-86.438423,32.458832,-86.422661,1095.65,3378.0,...,4.317376e+06,2.930909e+07,1.692559e+07,1.866793e+07,2.059855e+07,17536.774170,8.981854e+07,"[[0,151],[5,737],[10,1047],[15,758],[20,345],[...",53404.855511,8.0


Originally, merging was attempted using an if statement nested within a for loop which was nested in another for loop. Then the sunroofData and the censusData dataframes were merged using the 'merge' command. This massively improved the time that it took to combine the two dataframes. 

In [13]:
start = time.time()
combined = pd.merge(sunroofData,censusData)
end = time.time()
total = end-start
print('Operation took', total, ' seconds.')
combined.head()

Operation took 0.13992023468017578  seconds.


,tractID,state_name,lat_max,lat_min,lng_max,lng_min,lat_avg,lng_avg,yearly_sunlight_kwh_kw_threshold_avg,count_qualified,...,install_size_kw_buckets_json,carbon_offset_metric_tons,existing_installs_count,TotalPopulation,White,Black,Asian,Hispanic,Income,NAME
0,4.013010e+09,Arizona,34.037331,33.464901,-111.039902,-112.065399,33.743378,-111.546242,1365.1,2944.0,...,"[[0,169],[5,333],[10,367],[15,397],[20,342],[2...",52048.893549,134.0,4602.0,4408.0,25.0,134.0,59.0,115725.0,"Census Tract 101.02, Maricopa County, Arizona"
1,4.013030e+09,Arizona,33.853031,33.791840,-111.891182,-111.964851,33.816788,-111.916748,1365.1,2917.0,...,"[[0,275],[5,456],[10,422],[15,405],[20,311],[2...",47782.648106,105.0,4539.0,4472.0,0.0,62.0,170.0,113889.0,"Census Tract 304.01, Maricopa County, Arizona"
2,4.013030e+09,Arizona,33.900558,33.799080,-111.891251,-111.995979,33.860039,-111.936821,NaN,0.0,...,"[[0,281],[5,524],[10,494],[15,373],[20,276],[2...",0.000000,0.0,4243.0,4031.0,17.0,0.0,121.0,81994.0,"Census Tract 304.02, Maricopa County, Arizona"
3,4.013030e+09,Arizona,33.900558,33.799080,-111.891251,-111.995979,33.860039,-111.936821,1365.1,2574.0,...,"[[0,281],[5,524],[10,494],[15,373],[20,276],[2...",32251.907799,120.0,4243.0,4031.0,17.0,0.0,121.0,81994.0,"Census Tract 304.02, Maricopa County, Arizona"
4,4.013041e+09,Arizona,33.672272,33.638569,-112.324783,-112.358727,33.657543,-112.338348,1365.1,3261.0,...,"[[0,18],[5,226],[10,684],[15,887],[20,656],[25...",51711.565712,0.0,5404.0,5306.0,27.0,0.0,17.0,40978.0,"Census Tract 405.06, Maricopa County, Arizona"


More data cleaning is done in the next cell, namely some empty cells in the total number of panels column are removed. The method for doing that is replacing the blank cells with NaNs and then removing the rows with NaN in the panel number column. 

In [14]:
combined['number_of_panels_total'].replace('', np.nan,inplace=True)
combined.dropna(subset=['number_of_panels_total'], inplace=True)

To simplify the dataframe, the following cell eliminates the variables that are not relevant; this just makes it easier to view the relevant data. 

In [15]:
combined.drop(columns = ['lat_max', 'lat_min', 'lng_max', 'lng_min', 'lat_avg', 'lng_avg', 'yearly_sunlight_kwh_kw_threshold_avg', 'count_qualified', 'percent_covered', 'percent_qualified', 'number_of_panels_n', 'number_of_panels_s', 'number_of_panels_e', 'number_of_panels_w', 'number_of_panels_f', 'number_of_panels_median', 'kw_median', 'kw_total', 'yearly_sunlight_kwh_n', 'yearly_sunlight_kwh_s', 'yearly_sunlight_kwh_e', 'yearly_sunlight_kwh_w', 'yearly_sunlight_kwh_f', 'yearly_sunlight_kwh_median', 'yearly_sunlight_kwh_total', 'install_size_kw_buckets_json', 'carbon_offset_metric_tons', 'existing_installs_count', 'NAME']).head()

,tractID,state_name,number_of_panels_total,TotalPopulation,White,Black,Asian,Hispanic,Income
0,4.013010e+09,Arizona,301364.0,4602.0,4408.0,25.0,134.0,59.0,115725.0
1,4.013030e+09,Arizona,276815.0,4539.0,4472.0,0.0,62.0,170.0,113889.0
3,4.013030e+09,Arizona,186557.0,4243.0,4031.0,17.0,0.0,121.0,81994.0
4,4.013041e+09,Arizona,298081.0,5404.0,5306.0,27.0,0.0,17.0,40978.0
5,4.013041e+09,Arizona,340237.0,6395.0,6213.0,69.0,63.0,29.0,42378.0


## Next Steps

My next steps will include using Geopandas to visualize the demographic data in a select few counties, likely the largest counties, by census tract, as well as more exploratory data analysis.  

Then I will hopefully use some of the topics that we learned about more complex data science concepts like random forests to find out the most significant reason for uptake in solar power. 